<a href="https://colab.research.google.com/github/bshongwe/Other-FCC-Solutions/blob/master/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-11 08:04:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  96.0MB/s    in 0.3s    

2025-06-11 08:04:31 (96.0 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

# Complete the ratings dataframe loading (if not already done)
df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# Explore the data
print("Books dataset shape:", df_books.shape)
print("Ratings dataset shape:", df_ratings.shape)
print("\nFirst few books:")
print(df_books.head())
print("\nFirst few ratings:")
print(df_ratings.head())

Books dataset shape: (271379, 3)
Ratings dataset shape: (1149780, 3)

First few books:
         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  

First few ratings:
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0


In [4]:
# add your code here - consider creating a new cell for each section of code
# Data cleaning and filtering
print("Original ratings shape:", df_ratings.shape)

# Remove users with less than 200 ratings
user_rating_counts = df_ratings['user'].value_counts()
users_200_plus = user_rating_counts[user_rating_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(users_200_plus)]

print("After filtering users with <200 ratings:", df_ratings_filtered.shape)

# Remove books with less than 100 ratings
book_rating_counts = df_ratings_filtered['isbn'].value_counts()
books_100_plus = book_rating_counts[book_rating_counts >= 100].index
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(books_100_plus)]

print("After filtering books with <100 ratings:", df_ratings_filtered.shape)

Original ratings shape: (1149780, 3)
After filtering users with <200 ratings: (527556, 3)
After filtering books with <100 ratings: (13793, 3)


In [5]:
# Merge the datasets and create the book-user matrix
df_book_ratings = df_ratings_filtered.merge(df_books, on='isbn')

# Create pivot table: books as rows, users as columns, ratings as values
book_user_matrix = df_book_ratings.pivot_table(
    index='title',
    columns='user',
    values='rating'
).fillna(0)

print("Book-user matrix shape:", book_user_matrix.shape)

# Convert to sparse matrix for memory efficiency
book_user_sparse = csr_matrix(book_user_matrix.values)

# Initialize and fit the KNN model
model_knn = NearestNeighbors(
    metric='cosine',
    algorithm='brute',
    n_neighbors=6,  # 6 because we'll exclude the input book itself
    n_jobs=-1
)
model_knn.fit(book_user_sparse)

print("KNN model trained successfully!")

Book-user matrix shape: (99, 857)
KNN model trained successfully!


In [6]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    try:
        # Get the index of the book in our matrix
        book_idx = book_user_matrix.index.get_loc(book)

        # Get the book's rating vector
        book_vector = book_user_matrix.iloc[book_idx].values.reshape(1, -1)

        # Find nearest neighbors
        distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

        # Build recommendations list (skip index 0 as it's the book itself)
        recommendations = []
        for i in range(1, 6):  # Get 5 recommendations
            recommended_book = book_user_matrix.index[indices.flatten()[i]]
            distance = distances.flatten()[i]
            recommendations.append([recommended_book, distance])

        recommended_books = [book, recommendations]
        return recommended_books

    except KeyError:
        print(f"Book '{book}' not found in filtered dataset")
        return [book, []]
    except Exception as e:
        print(f"An error occurred: {e}")
        return [book, []]

In [7]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.7234864)], ["The Pilot's Wife : A Novel", np.float32(0.81926787)], ['The Joy Luck Club', np.float32(0.81986046)], ['The Notebook', np.float32(0.82366824)], ['Bel Canto: A Novel', np.float32(0.8247875)]]]
You haven't passed yet. Keep trying!
